## CNN sentence classifier - Rerun of training using only the first 21 tokens - the average sentence length.

In [1]:
## Enable Tensorboard
%load_ext tensorboard


## Usual Imports
import numpy as np

## Add local module folder to the path and load modules

import sys
sys.path.insert(0, '../python')
import debug
from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model



In [2]:
### The following uses compatibility mode to allow use
### of CUDA 11.2 with GPU support

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# maximum number of tokens to look at.
max_len = 21


### Read in the data set of sentences into numpy array.

In [4]:
d = load_data("../data/3xNCS.json")


Loaded 11056 data records.


## Split into train, validation and test sets

Before separating inputs and labels, duplicate the positive cases twice in the training data set.

In [5]:
%%time 

train_len = int(0.8 * len(d))
val_len = int(0.2 * len(d))

train, val = np.split(d, [train_len])

pos_samples = np.array( [ i for i in d if i["label"] == 1 ])

train = np.concatenate( (train, pos_samples, pos_samples) )

# Re-randomize the training array with the additional entries
np.random.seed(42)
np.random.shuffle(train)


# split into x and y sets for use in keras runs
x_train, y_train = np.array( [ i["text"] for i in train ] ), np.array( [ i["label"] for i in train ] )
x_val,   y_val   = np.array( [ i["text"] for i in val ] ),   np.array( [ i["label"] for i in val ]   )
# x_test,  y_test  = np.array( [ i["text"] for i in test ] ),  np.array( [ i["label"] for i in test ]  )


CPU times: user 13.5 ms, sys: 8.41 ms, total: 21.9 ms
Wall time: 21.2 ms


## Tokenize the training data set and build a vocabulary from it


In [6]:
# Tokenize the training data set and build a vocabulary from it
# Want to do this to minimize the vocab to just what is in
# the training set for performance.

x_train_ids, tokenizer = tokenize_sentences(x_train, max_len)
x_val_ids, tokenizer   = tokenize_sentences(x_val, max_len)

Loading previously created Tokenizer
Loading previously created Tokenizer


In [7]:
# Check for % of statements that are checkable claims to 
# ensure we have some level of balance that will avoid
# high accuracy for single value guessing.

print(f"proportion of checkable claims in training data  : {np.count_nonzero(y_train == 1)/len(y_train):.4f}")
print(f"proportion of checkable claims in validation data: {np.count_nonzero(y_val == 1)/len(y_val):.4f}")
# print(f"proportion of checkable claims in test data      : {np.count_nonzero(y_test == 1)/len(y_test):.4f}")

proportion of checkable claims in training data  : 0.5395
proportion of checkable claims in validation data: 0.2432


In [8]:
%%time

# Get the embedding matrix for the Embedding layer in Keras.
vocab_list, embedding_matrix, vocab_size = embed_matrix(tokenizer)

Loading previously created Embedding Matrix
CPU times: user 0 ns, sys: 2.7 ms, total: 2.7 ms
Wall time: 2.23 ms


In [9]:
len( embedding_matrix)

8196

In [10]:
embedding_matrix.shape[1]

50

In [11]:
print(len(x_train_ids))
print(len(y_train))
print(len(x_val_ids))
print(len(y_val))

14372
14372
2212
2212


In [12]:
# from jbyrne_utils import load_data, tokenize_sentences, embed_matrix, run_model

# run_model(embedding_matrix, x_train_ids, y_train, x_val_ids, y_val)

## Output of the tests are deleted as the notebook will not load in Chrome

In [13]:
%%time
# this is very quick, so we can use grid search on these models.

run_logfile = "runs2ballen21.log"

epochs = 20  # Not a variable, so we can determine the best epoch for the best model

#Write the header line
with open(run_logfile,'a') as f:
   f.write("timestamp|max_len|epochs|batch_size|embed_dim|num_filters|kernel_sizes|dense_layer_dims|dropout_rate|")
   for metric in [ 'loss', 'accuracy', 'val_loss', 'val_accuracy']:
       f.write(f"{metric}|")
       for i in range(0,epochs):
           f.write(f"{metric}-{i}|")
   f.write(f"END\n")        
   f.close()

    
# run the models  (Removed the filter sizes 32 or 64 that exceed the size of the shorter sequence)
for k_sizes in [ [8,16], [4,8,12], [4,8,16], [8,12,16] ]:

    # Set filter counts according to the number of kernels in use.
    if len(k_sizes) == 2:
        filter_counts = [ [16,16], [16,32], [32,32] ]
    if len(k_sizes) == 3:
        filter_counts = [ [16,16,16], [32, 32, 32], [64, 64, 64], [96, 96 ,96] ]
    if len(k_sizes) == 4:
        filter_counts = [ [2,2,2,2], [4,4,4,4], [8,8,8,8], [32,32,32,32], [128, 128, 128, 128], [8,16,32,64] ]
    
    
    for n_fil in filter_counts:
        
        for dld in [ [8], [16], [32], [32, 16] ]:

            run_model(embedding_matrix,
                      x_train_ids,
                      y_train,
                      x_val_ids,
                      y_val,
                      kernel_sizes     = k_sizes,
                      num_filters      = n_fil,
                      dense_layer_dims = dld,
                      epochs           = epochs,
                      logfile          = run_logfile,
                      logtag           = "tblogs-bal21",
                      model_dir        = "../models/CNNBal21",
                      max_len          = max_len)
            
            print("\n\n\n##########################################################################\n\n\n")



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 16
Epoch 1/20
288/288 [==============================] - 4s 7ms/step - loss: 0.6264 - accuracy: 0.6484 - val_loss: 0.6058 - val_accuracy: 0.6578
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-001445/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4427 - accuracy: 0.7996 - val_loss: 0.4512 - val_accuracy: 0.7884
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-001445/assets
Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.2882 - accuracy: 0.8820 - val_loss: 0.2671 - val_accuracy: 0.9001
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-001445/assets
Epoch 4/20
288/288 [==============================] - 1s 5ms/step - loss: 0.1842 - accuracy: 0.9342 - val_loss: 0.1972 - val_accuracy: 0.9259
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-001445/assets
Epoch 5/20
288/288 [=====

288/288 [==============================] - 1s 5ms/step - loss: 0.0916 - accuracy: 0.9686 - val_loss: 0.3157 - val_accuracy: 0.8969
Epoch 6/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0518 - accuracy: 0.9837 - val_loss: 0.2670 - val_accuracy: 0.9286
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-001553/assets
Epoch 7/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0371 - accuracy: 0.9892 - val_loss: 0.3988 - val_accuracy: 0.9005
Epoch 8/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0312 - accuracy: 0.9897 - val_loss: 0.2827 - val_accuracy: 0.9322
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-001553/assets
Epoch 9/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0295 - accuracy: 0.9898 - val_loss: 0.2562 - val_accuracy: 0.9390
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-001553/assets
Epoch 10/20
288/288 [==============================] - 1s 5ms/step - loss: 0

288/288 [==============================] - 1s 5ms/step - loss: 0.0111 - accuracy: 0.9969 - val_loss: 0.5016 - val_accuracy: 0.8938
Epoch 13/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0115 - accuracy: 0.9958 - val_loss: 0.3282 - val_accuracy: 0.9331
Epoch 14/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0135 - accuracy: 0.9954 - val_loss: 0.4299 - val_accuracy: 0.9182
Epoch 15/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0143 - accuracy: 0.9954 - val_loss: 0.3875 - val_accuracy: 0.9290
Epoch 16/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0100 - accuracy: 0.9961 - val_loss: 0.3892 - val_accuracy: 0.9349
Epoch 17/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0090 - accuracy: 0.9972 - val_loss: 0.5527 - val_accuracy: 0.9096
Epoch 18/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0076 - accuracy: 0.9969 - val_loss: 0.7494 - val_accuracy: 0.8766
Epoch 19/20

288/288 [==============================] - 1s 5ms/step - loss: 0.0078 - accuracy: 0.9977 - val_loss: 0.4499 - val_accuracy: 0.9358



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 16
Adding Convolution: Kernel Size: 16, Filter Count: 32
Epoch 1/20
288/288 [==============================] - 2s 6ms/step - loss: 0.6334 - accuracy: 0.6473 - val_loss: 0.5341 - val_accuracy: 0.7319
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-001837/assets
Epoch 2/20
288/288 [==============================] - 1s 5ms/step - loss: 0.4146 - accuracy: 0.8147 - val_loss: 0.3280 - val_accuracy: 0.8594
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-001837/assets
Epoch 3/20
288/288 [==============================] - 1s 5ms/step - loss: 0.2364 - accuracy: 0.9036 - val_loss: 0.2146 - val_accuracy: 0.9200
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-001837/assets
Epoch 4/20
288/288 [==========

Epoch 5/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0667 - accuracy: 0.9782 - val_loss: 0.2880 - val_accuracy: 0.9146
Epoch 6/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0328 - accuracy: 0.9915 - val_loss: 0.2812 - val_accuracy: 0.9222
Epoch 7/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0219 - accuracy: 0.9930 - val_loss: 0.3824 - val_accuracy: 0.9051
Epoch 8/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0342 - accuracy: 0.9872 - val_loss: 0.2831 - val_accuracy: 0.9349
Epoch 9/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0179 - accuracy: 0.9945 - val_loss: 0.4652 - val_accuracy: 0.8983
Epoch 10/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0158 - accuracy: 0.9948 - val_loss: 0.2885 - val_accuracy: 0.9367
Epoch 11/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0137 - accuracy: 0.9963 - val_loss: 0.5621 - val_accuracy: 0.8960
Epoc

...

288/288 [==============================] - 2s 5ms/step - loss: 0.0230 - accuracy: 0.9945 - val_loss: 0.2270 - val_accuracy: 0.9340
Epoch 8/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0159 - accuracy: 0.9956 - val_loss: 0.3452 - val_accuracy: 0.9114
Epoch 9/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0156 - accuracy: 0.9966 - val_loss: 0.2699 - val_accuracy: 0.9317
Epoch 10/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0111 - accuracy: 0.9977 - val_loss: 0.3143 - val_accuracy: 0.9277
Epoch 11/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0150 - accuracy: 0.9954 - val_loss: 0.3311 - val_accuracy: 0.9177
Epoch 12/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0112 - accuracy: 0.9972 - val_loss: 0.3560 - val_accuracy: 0.9195
Epoch 13/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0082 - accuracy: 0.9973 - val_loss: 0.4807 - val_accuracy: 0.9028
Epoch 14/20
2

288/288 [==============================] - 2s 5ms/step - loss: 0.0087 - accuracy: 0.9975 - val_loss: 0.2593 - val_accuracy: 0.9485
Epoch 15/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0173 - accuracy: 0.9941 - val_loss: 0.6209 - val_accuracy: 0.8978
Epoch 16/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0136 - accuracy: 0.9965 - val_loss: 0.4859 - val_accuracy: 0.9218
Epoch 17/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0062 - accuracy: 0.9975 - val_loss: 0.4903 - val_accuracy: 0.9164
Epoch 18/20
288/288 [==============================] - 1s 5ms/step - loss: 0.0078 - accuracy: 0.9974 - val_loss: 0.7083 - val_accuracy: 0.9060
Epoch 19/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0098 - accuracy: 0.9969 - val_loss: 0.4936 - val_accuracy: 0.9263
Epoch 20/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0090 - accuracy: 0.9964 - val_loss: 0.5686 - val_accuracy: 0.9064



########

288/288 [==============================] - 2s 7ms/step - loss: 0.6006 - accuracy: 0.6655 - val_loss: 0.3763 - val_accuracy: 0.8368
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-004619/assets
Epoch 2/20
288/288 [==============================] - 2s 5ms/step - loss: 0.3413 - accuracy: 0.8545 - val_loss: 0.2555 - val_accuracy: 0.8974
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-004619/assets
Epoch 3/20
288/288 [==============================] - 2s 5ms/step - loss: 0.1694 - accuracy: 0.9388 - val_loss: 0.1487 - val_accuracy: 0.9435
INFO:tensorflow:Assets written to: ../models/CNNBal21/210410-004619/assets
Epoch 4/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0641 - accuracy: 0.9838 - val_loss: 0.2541 - val_accuracy: 0.9168
Epoch 5/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0279 - accuracy: 0.9938 - val_loss: 0.3622 - val_accuracy: 0.8901
Epoch 6/20
288/288 [==============================] - 2s 5ms/step - loss: 0.

288/288 [==============================] - 2s 5ms/step - loss: 0.0232 - accuracy: 0.9936 - val_loss: 0.5005 - val_accuracy: 0.8807
Epoch 8/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0145 - accuracy: 0.9962 - val_loss: 0.4153 - val_accuracy: 0.9191
Epoch 9/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0168 - accuracy: 0.9949 - val_loss: 0.3312 - val_accuracy: 0.9335
Epoch 10/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0173 - accuracy: 0.9942 - val_loss: 0.4808 - val_accuracy: 0.9150
Epoch 11/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0084 - accuracy: 0.9977 - val_loss: 0.6126 - val_accuracy: 0.8924
Epoch 12/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0061 - accuracy: 0.9983 - val_loss: 0.4239 - val_accuracy: 0.9299
Epoch 13/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0410 - accuracy: 0.9869 - val_loss: 0.3677 - val_accuracy: 0.9331
Epoch 14/20
2

Epoch 16/20
288/288 [==============================] - 2s 5ms/step - loss: 0.0075 - accuracy: 0.9974 - val_loss: 0.3232 - val_accuracy: 0.9480
Epoch 17/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0150 - accuracy: 0.9949 - val_loss: 0.5704 - val_accuracy: 0.9173
Epoch 18/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0127 - accuracy: 0.9952 - val_loss: 0.5430 - val_accuracy: 0.9137
Epoch 19/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0017 - accuracy: 0.9997 - val_loss: 0.3389 - val_accuracy: 0.9471
Epoch 20/20
288/288 [==============================] - 2s 6ms/step - loss: 0.0095 - accuracy: 0.9974 - val_loss: 0.4939 - val_accuracy: 0.9204



##########################################################################



Adding Convolution: Kernel Size: 8, Filter Count: 96
Adding Convolution: Kernel Size: 12, Filter Count: 96
Adding Convolution: Kernel Size: 16, Filter Count: 96
Epoch 1/20
288/288 [=======================





### Dataset Citations

@inproceedings{arslan2020claimbuster,
    title={{A Benchmark Dataset of Check-worthy Factual Claims}},
    author={Arslan, Fatma and Hassan, Naeemul and Li, Chengkai and Tremayne, Mark },
    booktitle={14th International AAAI Conference on Web and Social Media},
    year={2020},
    organization={AAAI}
}

@article{meng2020gradient,
  title={Gradient-Based Adversarial Training on Transformer Networks for Detecting Check-Worthy Factual Claims},
  author={Meng, Kevin and Jimenez, Damian and Arslan, Fatma and Devasier, Jacob Daniel and Obembe, Daniel and Li, Chengkai},
  journal={arXiv preprint arXiv:2002.07725},
  year={2020}
}

Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib]


### Useful resources in building this
Using pre-trained word embeddings: https://keras.io/examples/nlp/pretrained_word_embeddings/